In [ ]:
import os
import asyncio
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_core.documents import Document
from core.metadata import Metadata
from core.document_loader import DocumentLoader
from core.document_summarizer import DocumentSummarizer
from langchain_core.prompts import ChatPromptTemplate

async def get_summary(llm: AzureChatOpenAI, docs: list[Document]) -> dict[str, any]:
    summarizer: DocumentSummarizer = DocumentSummarizer(llm, docs)
    
    app = summarizer.compile_app()

    step: dict = {}
    async for step in app.astream(
        {"contents": [doc.page_content for doc in docs]},
        {"recursion_limit": 10},
    ):
        pass

    return step

load_dotenv()

llm: AzureChatOpenAI = AzureChatOpenAI(
    model = "gpt-4o-mini",
    api_version = "2024-08-01-preview"
)

In [52]:
def extract_document(llm: AzureChatOpenAI, file_path: str):
    document_loader: DocumentLoader = DocumentLoader()
    docs: list[Document] = document_loader.load(file_path)

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an expert extraction algorithm. "
                "Only extract relevant information from the text. "
                "If you do not know the value of an attribute asked to extract, "
                "return null for the attribute's value.",
            ),
            ("human", "{text}"),
        ]
    )

    runnable = prompt | llm.with_structured_output(schema = Metadata)

    summary = asyncio.run(get_summary(llm, docs))
    metadata = runnable.invoke({"text": summary})

    return  metadata.title, metadata.keywords, summary['generate_final_summary']['final_summary']

In [59]:
import pandas as pd

df = pd.DataFrame(columns=['file_name', 'title', 'keywords', 'summary'])

for file_name in os.listdir(os.environ['DOCUMENT_PATH']):
    file_path = os.path.join(os.environ['DOCUMENT_PATH'], file_name)
    if os.path.isfile(file_path):
        title, keywords, summary = extract_document(llm, file_path)

        df = pd.concat([df, pd.DataFrame([{
            'file_name': file_name, 
            'title': title,
            'keywords': keywords,
            'summary': summary
        }])], ignore_index=True)

df.to_excel("output.xlsx")  

INFO: Preparing to split document for partition.
INFO: Starting page number set to 1
INFO: Allow failed set to 0
INFO: Concurrency level set to 5
INFO: Splitting pages 1 to 3 (3 total)
INFO: Determined optimal split size of 2 pages.
INFO: Partitioning 1 files with 2 page(s) each.
INFO: Partitioning 1 file with 1 page(s).
INFO: Partitioning set #1 (pages 1-2).
INFO: Partitioning set #2 (pages 3-3).
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: Successfully partitioned set #1, elements added to the final result.
INFO: Successfully partitioned set #2, elements added to the final result.
/Users/khanhvuongtuan/Desktop/Repo/Summarization/.venv/lib/python3.13/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/khanhvuongtuan/Desktop/Repo/Summarization/